In [1]:
import numpy as np
import cv2
import os
from matplotlib import pyplot as plt
from utils import *
import math
from tqdm import tqdm_notebook as tqdm

import Augmentor
import random

from shutil import copyfile

%matplotlib inline

### Setup
define the object directory

In [2]:
object_dir = "objects"

# names of the folders in objects/
categories = ['one_way_sign_left', 'one_way_sign_right', 'road_closed_sign', 'stop_sign', 'traffic_drum']

In [3]:
# Smooth edges on template images
# def smooth_borders(im):
#     h, w = im.shape[:2]
#     amt = 3
#     top_bot = np.zeros((3, w, 4))
#     top_bot[:] = (0, 0, 0, 0)
#     left_right = np.zeros((h+2*amt, 3, 4))
#     left_right[:] = (0, 0, 0, 0)

#     im = np.concatenate((top_bot, im, top_bot), axis=0)
#     im = np.concatenate((left_right, im, left_right), axis=1)

#     b, g, r, a = cv2.split(im)
#     a = cv2.GaussianBlur(a, (5,5), 5)
#     smoothened = cv2.merge((b, g, r, a))

#     return smoothened

# for cat in categories:
#     if not os.path.isdir(os.path.join("smooth", cat)):
#         os.mkdir(os.path.join("smooth", cat))
#     cat_path = os.path.join(object_dir, cat)
#     objs = os.listdir(cat_path)
#     for i, obj in enumerate(objs):
#         obj_path = os.path.join(cat_path, obj)
#         im = cv2.imread(obj_path, cv2.IMREAD_UNCHANGED)
#         smooth = smooth_borders(im)
#         cv2.imwrite(os.path.join("smooth", cat, "%s-%d.png" % (cat, i)), smooth)

### Step 2
Next, we will perform a random perspective transform as well as motion blur on each image and generate 50 samples per template image

In [4]:
# Iterate through object classes
if not os.path.isdir("temp"):
    os.mkdir("temp")
for cat in categories:
    now_playing(cat)
    if not os.path.isdir(os.path.join("temp", cat)):
        os.mkdir(os.path.join("temp", cat))
    # Iterate through images in classes
    curr_dir = os.path.join(object_dir, cat)
    cat_objs = os.listdir(curr_dir)
    im_idx=1
    for obj in tqdm(cat_objs):
        obj_path = os.path.join(curr_dir, obj)
        # Generate 50 samples from each object
        for _ in range(50):
            # Read image
            im = cv2.imread(obj_path, cv2.IMREAD_UNCHANGED)
            
            # Perspective transform
            if random.random() < 0.8:
                im = perspective_transform(im)
            
            # Motion blur
            if random.random() < 0.8:
                im = motion_blur(im, size=random.randint(3, 15))
            aug_path = "temp/%s/%s-%d.png" % (cat, cat, im_idx)
            cv2.imwrite(aug_path, im)
            im_idx += 1
print("done!")
#             # JPEG Compression (to generate artifacts)
#             im = compress_jpeg(im, min_quality=i*5, max_quality=i*5+5)

            # Salt and pepper noise. Some papers claim it helps, some claim it hurts. who really knows?
#             if(random.random() < 0.3):
#                 im = noisy(im, "s&p")

   ɴᴏᴡ ᴘʟᴀʏɪɴɢ: one_way_sign_left 
───────────────⚪─────────────────── 
◄◄⠀▐▐ ⠀►►⠀⠀ 1:17 / 3:48 ⠀ ───○ 🔊⠀ ᴴᴰ ⚙ ❐ ⊏⊐



   ɴᴏᴡ ᴘʟᴀʏɪɴɢ: one_way_sign_right 
───────────────⚪─────────────────── 
◄◄⠀▐▐ ⠀►►⠀⠀ 1:17 / 3:48 ⠀ ───○ 🔊⠀ ᴴᴰ ⚙ ❐ ⊏⊐



   ɴᴏᴡ ᴘʟᴀʏɪɴɢ: road_closed_sign 
───────────────⚪─────────────────── 
◄◄⠀▐▐ ⠀►►⠀⠀ 1:17 / 3:48 ⠀ ───○ 🔊⠀ ᴴᴰ ⚙ ❐ ⊏⊐



   ɴᴏᴡ ᴘʟᴀʏɪɴɢ: stop_sign 
───────────────⚪─────────────────── 
◄◄⠀▐▐ ⠀►►⠀⠀ 1:17 / 3:48 ⠀ ───○ 🔊⠀ ᴴᴰ ⚙ ❐ ⊏⊐



   ɴᴏᴡ ᴘʟᴀʏɪɴɢ: traffic_drum 
───────────────⚪─────────────────── 
◄◄⠀▐▐ ⠀►►⠀⠀ 1:17 / 3:48 ⠀ ───○ 🔊⠀ ᴴᴰ ⚙ ❐ ⊏⊐



done!


In [5]:
if not os.path.isdir("output"):
    os.mkdir("output")
for cat in categories:
    now_playing(cat)
    p = Augmentor.Pipeline("temp/"+cat, output_directory="../../output/"+cat)

    # Slightly distort picture. It should add a little variance to the objects
    p.random_distortion(probability=0.5, grid_width=4, grid_height=4, magnitude=1)

    # Change brightness, color, contrast
    p.random_brightness(probability=0.8, min_factor=0.6, max_factor=1)
    p.random_color(     probability=0.8, min_factor=0.6, max_factor=1)
    p.random_contrast(  probability=0.8, min_factor=0.5, max_factor=1)

    # Replace random section with noise
    p.random_erasing(probability=0.05, rectangle_area=0.65)
    p.random_erasing(probability=0.1,  rectangle_area=0.55)
    p.random_erasing(probability=0.3,  rectangle_area=0.35)
    p.random_erasing(probability=0.3,  rectangle_area=0.25)
    p.random_erasing(probability=0.3,  rectangle_area=0.15)

    # 2D rotation
    p.rotate_without_crop(probability=0.9, max_left_rotation=5, max_right_rotation=5, expand=True)
    p.rotate_without_crop(probability=0.2, max_left_rotation=5, max_right_rotation=5, expand=True)

    # Create 2000 new thumbnails for every class
    p.sample(2000)
    
    

Executing Pipeline:   0%|          | 0/2000 [00:00<?, ? Samples/s]

   ɴᴏᴡ ᴘʟᴀʏɪɴɢ: one_way_sign_left 
───────────────⚪─────────────────── 
◄◄⠀▐▐ ⠀►►⠀⠀ 1:17 / 3:48 ⠀ ───○ 🔊⠀ ᴴᴰ ⚙ ❐ ⊏⊐
Initialised with 1300 image(s) found.
Output directory set to temp/one_way_sign_left/../../output/one_way_sign_left.

Processing <PIL.Image.Image image mode=RGBA size=616x256 at 0x7F01643910B8>: 100%|██████████| 2000/2000 [00:13<00:00, 153.45 Samples/s]
Executing Pipeline:   0%|          | 0/2000 [00:00<?, ? Samples/s]

   ɴᴏᴡ ᴘʟᴀʏɪɴɢ: one_way_sign_right 
───────────────⚪─────────────────── 
◄◄⠀▐▐ ⠀►►⠀⠀ 1:17 / 3:48 ⠀ ───○ 🔊⠀ ᴴᴰ ⚙ ❐ ⊏⊐
Initialised with 1250 image(s) found.
Output directory set to temp/one_way_sign_right/../../output/one_way_sign_right.

Processing <PIL.Image.Image image mode=RGBA size=383x150 at 0x7F0164502D30>: 100%|██████████| 2000/2000 [00:11<00:00, 181.60 Samples/s]
Executing Pipeline:   0%|          | 0/2000 [00:00<?, ? Samples/s]

...

Processing <PIL.Image.Image image mode=RGBA size=610x492 at 0x7F0164671E48>: 100%|██████████| 2000/2000 [00:14<00:00, 135.38 Samples/s]
Executing Pipeline:   0%|          | 0/2000 [00:00<?, ? Samples/s]

   ɴᴏᴡ ᴘʟᴀʏɪɴɢ: stop_sign 
───────────────⚪─────────────────── 
◄◄⠀▐▐ ⠀►►⠀⠀ 1:17 / 3:48 ⠀ ───○ 🔊⠀ ᴴᴰ ⚙ ❐ ⊏⊐
Initialised with 1050 image(s) found.
Output directory set to temp/stop_sign/../../output/stop_sign.

Processing <PIL.Image.Image image mode=RGBA size=632x623 at 0x7F0164666BA8>: 100%|██████████| 2000/2000 [00:17<00:00, 112.93 Samples/s]
Executing Pipeline:   0%|          | 0/2000 [00:00<?, ? Samples/s]

   ɴᴏᴡ ᴘʟᴀʏɪɴɢ: traffic_drum 
───────────────⚪─────────────────── 
◄◄⠀▐▐ ⠀►►⠀⠀ 1:17 / 3:48 ⠀ ───○ 🔊⠀ ᴴᴰ ⚙ ❐ ⊏⊐
Initialised with 950 image(s) found.
Output directory set to temp/traffic_drum/../../output/traffic_drum.

Processing <PIL.Image.Image image mode=RGBA size=257x416 at 0x7F016453B630>: 100%|██████████| 2000/2000 [00:13<00:00, 148.15 Samples/s] 


now we paste the augmented objects onto background images

In [9]:
# Get paths of all backgrounds
BG_PATH = os.path.join("backgrounds", "resize")
background_paths = [os.path.join(BG_PATH, bg_path) for bg_path in os.listdir(BG_PATH)]
random.shuffle(background_paths)

cat_paths = [os.path.join("output", cat) for cat in categories]

# Get paths of all generated objects
obj_paths = []
for cat in categories:
    cat_path = os.path.join("output", cat)
    for obj_path in os.listdir(cat_path):
        obj_paths.append({
            "category": cat,
            "path": os.path.join(cat_path, obj_path)
        })

# Shuffle object paths
random.shuffle(obj_paths)

In [10]:
if not os.path.exists("generated"):
    os.mkdir("generated")
curr_obj_idx = 0

for bg_idx, bg_path in enumerate(background_paths):
    if bg_idx % 500 == 0:
        print(len(background_paths) - bg_idx, " images left")
        
    # Check if we've gone through all the objects
    if bg_idx > 10000:
        remaining_bgs = background_paths[bg_idx:]
        print("Copying over the remaining", len(remaining_bgs), "images...")
        for rem_idx, rem_bg_path in enumerate(remaining_bgs):
            new_bg_path = "generated/rem-%d.jpg" % rem_idx
            bbox_path = "generated/rem-%d.txt" % rem_idx
            
            # Copy image to generated folder
            copyfile(rem_bg_path, new_bg_path)
            
            # Create empty annotation file
            open(bbox_path, 'w').close()
        break
        
    background = cv2.imread(bg_path)
    bg_h, bg_w = background.shape[:2]
    
    # Generate new path names
    new_bg_path = "generated/img-%d.jpg" % bg_idx
    bbox_path = "generated/img-%d.txt" % bg_idx
    
    # Open bounding box file
    bbox_file = open(bbox_path, 'w')
    
    # Place between 1 and 5 objects on each image
    num_objs = random.randint(1, 5)
    ignore_bboxes = []
    for _ in range(num_objs):
        # If we've gone through all our objects, shuffle and go again
        if curr_obj_idx >= len(obj_paths):
            random.shuffle(obj_paths)
            curr_obj_idx = 0
        obj_cat = obj_paths[curr_obj_idx]["category"]
        obj_path = obj_paths[curr_obj_idx]["path"]
        
        obj = cv2.imread(obj_path, cv2.IMREAD_UNCHANGED)
        
        background, bbox = paste_random(background, obj, ignore_bboxes)
        ignore_bboxes.append(bbox)
        
        x1, y1 = bbox[0]
        x2, y2 = bbox[1]
        
        bbox_file.write("%d %d %d %d %d %d %s\n" % (x1, y1, x2, y2, bg_w, bg_h, obj_cat))
        curr_obj_idx += 1
    bbox_file.close()
    cv2.imwrite(new_bg_path, background)
print("done!")
        
    
    

18686  images left
18186  images left
17686  images left
17186  images left
16686  images left
16186  images left
15686  images left
15186  images left
14686  images left
14186  images left
13686  images left
13186  images left
12686  images left
12186  images left
11686  images left
11186  images left
10686  images left
10186  images left
9686  images left
9186  images left
8686  images left
Copying over the remaining 8685 images...
done!
